In [1]:
# mount drive and add path to root folder so custom_GANs python package is available
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/My Drive/generative_models')


Mounted at /content/drive


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [3]:
from pathlib import Path
import os

import tensorflow as tf
import keras
import numpy as np

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

from src.custom_GANs.simple_GANs import ConditionalGAN, ModelMonitor, CheckpointCleanupCallback, GANCheckpoint, ConditionalGenerator, ConditionalDiscriminator


In [4]:
batch_size = 64
epochs = 100
num_channels = 1
num_example_images = 3
num_classes = 10
image_size = 28
latent_dim = 128
output_dim = (image_size, image_size, 1)


In [5]:
def scale_images(images, labels):
    image = tf.cast(images, tf.float32)
    # Scale the image
    image = (image - 127.5) / 127.5
    return image, labels

In [6]:
# We'll use all the available examples from both the training and test
# sets.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_labels = np.concatenate([y_train, y_test])
all_labels = all_labels.reshape(-1, 1)


In [7]:
#debug
"""subset_size = 50  # Choose a small number, like 1000 examples
all_digits = all_digits[:subset_size]
all_labels = all_labels[:subset_size]"""


'subset_size = 50  # Choose a small number, like 1000 examples\nall_digits = all_digits[:subset_size]\nall_labels = all_labels[:subset_size]'

In [8]:
# Scale the pixel values to [0, 1] range, add a channel dimension to
# the images, and one-hot encode the labels.
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))

# Create tf.data.Dataset.
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.map(scale_images)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")

Shape of training images: (70000, 28, 28, 1)
Shape of training labels: (70000, 1)


In [9]:
example_images_dir =  Path('/content/drive/My Drive/generative_models').resolve() / 'examples/images/CGAN'
example_images_dir.mkdir(parents=True, exist_ok=True)
print('Example images save dir: ', example_images_dir)

checkpoint_dir =  Path('/content/drive/My Drive/generative_models').resolve() / 'examples/checkpoints/CGAN'
checkpoint_dir.mkdir(parents=True, exist_ok=True)
print('Checkpoint save dir: ', checkpoint_dir)

Example images save dir:  /content/drive/My Drive/generative_models/examples/images/CGAN
Checkpoint save dir:  /content/drive/My Drive/generative_models/examples/checkpoints/CGAN


In [10]:
checkpoint_callback = GANCheckpoint(checkpoint_dir=checkpoint_dir)

noise_generator = lambda: tf.random.uniform((num_example_images, latent_dim,1))
label_generator = lambda: tf.random.uniform((num_example_images, 1), minval=0, maxval=num_classes, dtype=tf.int32 )

generator_input_generators = [noise_generator, label_generator]
checkpoint_model_monitor = ModelMonitor(save_path = example_images_dir, num_img = num_example_images, input_generators = generator_input_generators)


In [11]:
cgan = ConditionalGAN(latent_dim=latent_dim, output_dim=output_dim, nr_classes=num_classes)
cgan.compile()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
cgan.fit(dataset, epochs = epochs, batch_size = batch_size, callbacks = [checkpoint_model_monitor, checkpoint_callback, CheckpointCleanupCallback(checkpoint_dir)])

Epoch 1/100
547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - d_loss: 0.2880 - g_loss: 2.4136ModelMonitor: Saving image to  /content/drive/My Drive/generative_models/examples/images/CGAN

Epoch 1: g_loss improved from inf to 2.6705, saving model
Deleted old checkpoint: discriminator_epoch_01_g_loss_2.67.keras
Deleted old checkpoint: generator_epoch_01_g_loss_2.67.keras
547/547 ━━━━━━━━━━━━━━━━━━━━ 67s 97ms/step - d_loss: 0.2879 - g_loss: 2.4141
Epoch 2/100
547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - d_loss: 0.2677 - g_loss: 2.5755ModelMonitor: Saving image to  /content/drive/My Drive/generative_models/examples/images/CGAN

Epoch 2: g_loss improved from 2.6705 to 2.4364, saving model
Deleted old checkpoint: discriminator_epoch_02_g_loss_2.44.keras
Deleted old checkpoint: generator_epoch_02_g_loss_2.44.keras
547/547 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - d_loss: 0.2677 - g_loss: 2.5752
Epoch 3/100
547/547 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - d_loss: 0.2680 - g_loss: 2.5772ModelMonitor: Saving ima